<a href="https://colab.research.google.com/github/Nohalyan/Projetppchem/blob/main/Notebook_WSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Water Solubility Predisction

This notebook is the core of our Water Solubility Prediction project. This is where we collected, sorted and cleaned the data, as well as selecting, testing and training our prediction models.

This notebook is divided into sections:
1. Modules and Libraries
2. Solubility Data *(optional)*
3. Calculation of RDkit Molecular Descriptors *(optional)*
4. Training and Validation set
5. Select Machine Learning Models
6. Fine-tuning
7. Models for training and test data

Sections 2 and 3 are optional, as sorted and cleaned data can be retrieved directly in section 4.1.


## 1 Import Relevant Modules and Libraries

We will start by importing the modules and libraries essential for this project.


In [ ]:
# Install all libraries used in this project
!pip install pathlib numpy pandas rdkit matplotlib scikit-learn lightgbm lazypredict tqdm

In [2]:
# Install all modules used in this project
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

import numpy as np

import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

from matplotlib import pyplot as plt
import matplotlib.patches as mpatches

import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

from lightgbm import LGBMRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

import lazypredict
from lazypredict.Supervised import LazyRegressor

from tqdm import tqdm



# 2. Solubility Data

## 2.1 Let's get the Solubility Data

Firstly, we will acquire solubility data from gashawmg's repository, accessible at https://github.com/gashawmg. Subsequently, we will conduct an exploratory data analysis on the acquired dataset.

In [ ]:
!wget https://raw.githubusercontent.com/Nohalyan/Projetppchem/main/Data/Data_Solubility.csv

Let's open the file ans confirm that it exist.

In [ ]:
# Create a Path object for the current directory, in our case /content/
current_directory = Path.cwd()
print("Current Directory:", current_directory.resolve())

file_path = current_directory / "Data_Solubility.csv"

# Reading the contents of the file and check that the file exists
if file_path.exists():
    with file_path.open("r") as file:
        content = file.read()
        print(content)
else:
    print("The file does not exist.")


The file use semicicolon as delimiter, so let's open the file and use semicicolon as delimiter:

In [6]:
# Open a file containing descriptors and yield
data_solubility = pd.read_csv("/content/Data_Solubility.csv", delimiter=';')

Now we will check the data see if it is what we want.

In [ ]:
data_solubility.shape

In [ ]:
data_solubility.head()

## 2.2 Data Cleaning

Now that we've got our data set, we need to clean it up. To do this, we're going to remove non-numericals (NaN) and null values. We'll also sort outliers and make SMILES canonical, so as to duplicate them and obtain a ready-to-use data set.

### 2.2.1 Remove NaN or null values

We wil start by removing non-numerical values and valeurs that are null:

In [9]:
data_solubility.SMILES.isnull().sum()
data_solubility.dropna(inplace=True)
data_solubility.shape

(9943, 6)

As we can see, the shape is still the same, the data has already been cleaned of non-numerical and null values.

###2.2.2 Remove outliers

Then, we will remove outliers from the data. Using a boxplot, we can easely visualize outliers:


In [ ]:
sn.set_theme()
sn.displot(data=data_solubility, x="logS", binwidth=1)

We will then filter compounds that follow as close as normal distribution. We consider values between -7.5 and 1.7 as following a normal distribution.


In [11]:
new_data_solubility = data_solubility[data_solubility.logS.apply(lambda x: x > -7.5 and x < 1.7)]

We will generate a histogram to visualize the distribution of the new data.

In [ ]:
sn.displot(data=new_data_solubility, x='logS', binwidth=1,kde=True)
new_data_solubility.shape

### 2.2.3 Remove Duplicates

Then, we will remove duplicate by generating canonical SMILES.

In [13]:
# Generate a canonical SMILES function
def canonical_SMILES(smiles):
    canon_smls = [Chem.CanonSmiles(smls) for smls in smiles]
    return canon_smls

In [ ]:
# Generate canonical Smiles using the function
canon_smiles = canonical_SMILES(new_data_solubility.SMILES)

# Replace SMILES column with canonical SMILES
new_data_solubility["SMILES"] = canon_smiles

# Create a list for duplicate smiles
duplicate_smiles = new_data_solubility[new_data_solubility['SMILES'].duplicated()]['SMILES'].values
len(duplicate_smiles)

Upon analysis,we found out that there are six duplicate entries. These duplicates will be filtered out and the dataset will be sorted for improved readability.

In [ ]:
new_data_solubility[new_data_solubility['SMILES'].isin(duplicate_smiles)].sort_values(by=['SMILES'])

We removed rows containing duplicate SMILES, retaining the first occurrence of each structure.

In [ ]:
data_solubility_cleaned = new_data_solubility.drop_duplicates(subset=['SMILES'], keep='first')
data_solubility_cleaned.shape
data_solubility_cleaned.head()

### 2.2.4 Filter training data
With the dataset in hand, our next step involves assembling a test set comprising 100 drug-like compounds. These compounds were sourced from https://github.com/PatWalters/solubility.

In [ ]:
!wget https://raw.githubusercontent.com/Nohalyan/Projetppchem/main/Data/Data_Drug_Like_Solubility.csv

In [ ]:
# Create a Path object for the current directory, in our case /content/
current_directory_dl = Path.cwd()
print("Current Directory:", current_directory.resolve())

file_path_dl = current_directory / "Data_Drug_Like_Solubility.csv"

# Reading the contents of the file and check that the file exists
if file_path.exists():
    with file_path.open("r") as file:
        content = file.read()
#        print(content)
else:
    print("The file does not exist.")

In [ ]:
data_dl = pd.read_csv("/content/Data_Drug_Like_Solubility.csv", delimiter=';')
data_dl.shape

In [ ]:
data_dl.head()

In [ ]:
# Generate canonical Smiles
canon_smiles = canonical_SMILES(data_dl.SMILES)

# Replace SMILES column wit Canonical SMILES
data_dl["SMILES"] = canon_smiles

# Create a list for duplicate smiles
duplicate_data_dl_smiles = data_dl[data_dl['SMILES'].duplicated()]['SMILES'].values
len(duplicate_data_dl_smiles)

Now we need to check that there are no duplicates between the data set and the test data set.

In [ ]:
# Molecules used in training and test of the model
data_dl_SMILES = data_dl.SMILES.values

# Filter molecules that are not present in the test set
data_cleaned_final = data_solubility_cleaned[~data_solubility_cleaned['SMILES'].isin(data_dl_SMILES)]
print(f'Compounds present in training set:{len(data_solubility_cleaned) - len(data_cleaned_final)}')
data_cleaned_final.shape

In [23]:
# Save the dataframe to a CSV file
data_cleaned_final.to_csv('Data_Cleaned_Final.csv', index=False)

In [ ]:
data_dl= data_dl[data_dl['LogS exp (mol/L)'].apply(lambda x: x > -7.5 and x < 1.7)]
data_dl

# 3. Calculation of RDkit Molecular Descriptors

Calculation of RDkit Molecular Descriptors, which are molecular features for the data set and the test data set

In [ ]:
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0]
                                    for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    Mol_descriptors =[]
    for mol in tqdm(mols):
        # add hydrogens to molecules
        mol=Chem.AddHs(mol)
        # Calculate all 200 descriptors for each molecule
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names

# Function call
Mol_descriptors,desc_names = RDkit_descriptors(data_cleaned_final['SMILES'])

In [ ]:
df_descriptors = pd.DataFrame(Mol_descriptors, columns=desc_names)
df_descriptors.head()

In [ ]:
# Calculate molecular descriptors for the test data or 98 compounds
Mol_descriptors_test , desc_names_test = RDkit_descriptors(data_dl["SMILES"])
data_dl_descriptors = pd.DataFrame(Mol_descriptors_test,columns=desc_names_test)

In [28]:
# Save the dataf to .csv files
df_descriptors.to_csv('Data_Solubility_descriptor.csv', index=False)
data_dl_descriptors.to_csv('Data_Solubility_Drug_Like_descriptor.csv', index=False)

# 4. Training and Validation set

In this part, we will separate the data from the data set into 2 sets: the training set to train the model and the validation set to provide an evaluation of the model on the training dataset to adjust the hyperparameters.

If you have run sections 2. and 3., you can skip to section 4.2, otherwise you must run section 4.1.

##4.1 Acquisition of Sorted, Cleaned Data Sets and RDkit Molecular Descriptors

In [ ]:
#Download cleaned and sorted data (Data_Cleaned_Final.csv) with RDkit molecular descriptors (Data_Solubility_descriptor.csv) for training and validation set
!wget https://raw.githubusercontent.com/Nohalyan/Projetppchem/main/Data/Data_Cleaned_Final.csv
!wget https://raw.githubusercontent.com/Nohalyan/Projetppchem/main/Data/Data_Solubility_descriptor.csv

#Download cleaned and sorted data (Data_Drug_Like_Solubility.csv) with RDkit molecular descriptors (Data_Solubility_Drug_Like_descriptor.csv) for test set
!wget https://raw.githubusercontent.com/Nohalyan/Projetppchem/main/Data/Data_Drug_Like_Solubility.csv
!wget https://raw.githubusercontent.com/Nohalyan/Projetppchem/main/Data/Data_Solubility_Drug_Like_descriptor.csv

In [ ]:
#To verify that the file was correctly downloaded
# Create a Path object for the current directory, in our case /content/
current_directory = Path.cwd()
print("Current Directory:", current_directory.resolve())

file_path = current_directory / "Data_Solubility_Drug_Like_descriptor.csv" #Copy and paste the file name

# Reading the contents of the file and check that the file exists
if file_path.exists():
    with file_path.open("r") as file:
        content = file.read()
        print(content)
else:
    print("The file does not exist.")


In [5]:
#Variable definitions, if not defined in sections 2. and 3.
data_dl = pd.read_csv("/content/Data_Drug_Like_Solubility.csv", delimiter=';')
data_dl= data_dl[data_dl['LogS exp (mol/L)'].apply(lambda x: x > -7.5 and x < 1.7)]
df_descriptors = pd.read_csv("/content/Data_Solubility_descriptor.csv", delimiter=';')
data_cleaned_final = pd.read_csv("/content/Data_Cleaned_Final.csv", delimiter=',')
data_dl_descriptors = pd.read_csv("/content/Data_Solubility_Drug_Like_descriptor.csv", delimiter=',')

## 4.2  Split the chemicals for training and validation set

In [6]:
#Variable definitions for training and set validation
x_train, x_valid, y_train, y_valid = train_test_split(df_descriptors, data_cleaned_final.logS, test_size=0.1,random_state=42)

#Standardization of the features
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_valid_scaled = scaler.transform(x_valid)

#5. Select Machine Learning Models

Let's identify the optimal machine learning models for our water solubility predisction. To accomplish this, we will utilize the "lazypredict" library, specifically leveraging the "LazyRegressor" function to evaluate a selection of 42 machine learning models.

In [ ]:
lregs = LazyRegressor(verbose=0,ignore_warnings=True, custom_metric=None,random_state=42)
models, prediction_tests = lregs.fit(x_train_scaled, x_valid_scaled, y_train, y_valid)

In [ ]:
#The top five models
prediction_tests[:5]

# 6. Fine-tuning

Analyzing the top 5 best models we decided to use 3 models:
1. LGBMRegressor,
2. XGBRegressor because these are the 2 fastest models, enabling us to obtain results faster than the other models.
3. RandomForestRegressor because we were familiar with this modeland we wanted to see what it could do in the project.

## 6.1 LGBMRegressor

We decided to take the LGBMRegressor model because the results generated by this model are comparable to the ExtraTreesRegressor model, but takes a lot less time than the ExtraTreesRegressor model.Let's performs a grid search using GridSearchCV from scikit-learn to find the best hyperparameters for a LightGBM regressor:

In [ ]:
params = {'max_depth' : list(range(20, 30, 1)),
          'n_estimators' : list(range(1278, 1280, 1)) ,
          'learning_rate': list(np.arange(0.04, 0.05, 0.01))}

grid_search = GridSearchCV(LGBMRegressor(random_state = 42),
                            param_grid=params, cv=5, verbose=1)

grid_search.fit(x_train, y_train)

print("Optimized parameters for a LightGBM regressor can be: ", grid_search.best_params_)

We obtained:
* learning_rate: 0.01
* max_depth: 26
* n_estmitors: 901

Let's optimize again with new ranges for max_depth, n_estmitors and learning_rate to obtain even better parameters:


In [ ]:
#params_bst = {"max_depth" : list(range(20, 36, 4)),
#         "n_estimators" : list(range(850, 1200, 50)),
#         "learning_rate" : list(np.arange(0.01, 0.05, 0.01))}
#
#grid_search_bst = GridSearchCV(LGBMRegressor(random_state = 42),
#                                  param_grid=params_bst, cv=3, verbose=1)
#
#grid_search_bst.fit(x_train, y_train)
#print("The best parameters are: ", grid_search_bst.best_params_)

We obtained:
* learning_rate: 0.04
* max_depth: 24
* n_estmitors: 1151

These are the parameters we will use for our standard LGBMRegressor model.

We also wanted to see if by making new runs and trying to optimize these three hyperparameters even further, we could get an even better model. We therefore made a total of 6 runs (compared with the 2 runs for standard models) for this model and obtained these parameters:
* learning_rate: 0.04
* max_depth: 21
* n_estmitors: 1279

These are the parameters we will use for our studious LGBMRegressor model.

##6.2 XGB Regressor
In order to compare our results obtained for the first model, we have chosen XGB Regressor as the second model for the reasons presented above.

In [ ]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming x_train and y_train are already defined

# Split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Data preprocessing
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)

# Define the parameter grid
max_depths = list(range(2, 32, 8))
n_estimators = list(range(1, 201, 20))
learning_rates = list(np.arange(0.01, 1.02, 0.25))

best_score = float('-inf')
best_params = {}

# Iterate over parameter combinations
for max_depth in max_depths:
    for n_estimator in n_estimators:
        for learning_rate in learning_rates:
            # Create XGBoost regressor with current parameters
            xgb_regressor = XGBRegressor(max_depth=max_depth,
                                         n_estimators=n_estimator,
                                         learning_rate=learning_rate,
                                         random_state=42)

            # Fit the model
            xgb_regressor.fit(x_train_scaled, y_train)

            # Evaluate the model
            score = xgb_regressor.score(x_val_scaled, y_val)  # Using the validation set for evaluation

            # Check if current parameters yield better score
            if score > best_score:
                best_score = score
                best_params = {'max_depth': max_depth, 'n_estimators': n_estimator, 'learning_rate': learning_rate}

# Print the best parameters found
print("Optimized parameters for an XGBoost regressor can be:", best_params)

We obtained for this first run:
- max_depth: 2
- n_estimators: 181
- learning_rate: 0.26

In [ ]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming x_train and y_train are already defined

# Split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Data preprocessing
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)

# Define the parameter grid
max_depths = list(range(2, 6, 1))
n_estimators = list(range(100, 700, 50))
learning_rates = list(np.arange(0.05, 0.30, 0.05))

best_score = float('-inf')
best_params = {}

# Iterate over parameter combinations
for max_depth in max_depths:
    for n_estimator in n_estimators:
        for learning_rate in learning_rates:
            # Create XGBoost regressor with current parameters
            xgb_regressor = XGBRegressor(max_depth=max_depth,
                                         n_estimators=n_estimator,
                                         learning_rate=learning_rate,
                                         random_state=42)

            # Fit the model
            xgb_regressor.fit(x_train_scaled, y_train)

            # Evaluate the model
            score = xgb_regressor.score(x_val_scaled, y_val)  # Using the validation set for evaluation

            # Check if current parameters yield better score
            if score > best_score:
                best_score = score
                best_params = {'max_depth': max_depth, 'n_estimators': n_estimator, 'learning_rate': learning_rate}

# Print the best parameters found
print("Optimized parameters for an XGBoost regressor can be:", best_params)


We obtained for this second run:

- max_depth: 5
- n_estimators: 500
- learning_rate: 0.1

These are the parameters we will use for our standard LGBMRegressor model.

## 6.3 RandomForestRegressor

For the third regression model, we opted for the RandomForestRegressor model. However, it presented longer runtimes, allowing us to explore its post-tuning performance and see how it differs from the other models.

In [ ]:
# Handle missing values in x_train
imputer = SimpleImputer(strategy='mean')
x_train_imputed = imputer.fit_transform(x_train)

# Scale the input data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_imputed)

# Define the parameter grid for grid search
paramsRF = {
    'max_depth': list(range(2, 32, 16)),
    'n_estimators': list(range(1, 1000, 200))
}

# Perform grid search with RandomForestRegressor
grid_searchRF = GridSearchCV(RandomForestRegressor(random_state=42),
                             param_grid=paramsRF, cv=5, verbose=1)

# Fit the grid search to the scaled data
grid_searchRF.fit(x_train_scaled, y_train)

# Print the best parameters found
print("Optimized parameters for a RandomForestRegressor can be: ", grid_searchRF.best_params_)

We acquired those values for the first run:
- max_depth = 26
- n_estimators = 901

In [ ]:
# Handle missing values in x_train
#imputer = SimpleImputer(strategy='mean')
#x_train_imputed = imputer.fit_transform(x_train)

# Scale the input data
#scaler = StandardScaler()
#x_train_scaled = scaler.fit_transform(x_train_imputed)

# Define the parameter grid for grid search
#paramsRF = {
#    'max_depth': list(range(2, 32, 12)),
#    'n_estimators': list(range(1, 1000, 150))
#}

# Perform grid search with RandomForestRegressor
#grid_searchRF = GridSearchCV(RandomForestRegressor(random_state=42),
#                             param_grid=paramsRF, cv=5, verbose=1)

# Fit the grid search to the scaled data
#grid_searchRF.fit(x_train_scaled, y_train)

# Print the best parameters found
#print("Optimized parameters for a RandomForestRegressor can be: ", grid_searchRF.best_params_)



We acquired those values for the first run:
- max_depth = 18
- n_estimators = 801

These are the parameters we will use for our studious RandomForestRegressor model.

# 7. Models for training and test data


## 7.1 LGBMRegressor
Let's test our model ont the training and test set with the best parameters found with the fine tunning:





In [ ]:
model = LGBMRegressor(n_estimators = 1279,
                      max_depth = 21,
                      learning_rate = 0.04,
                      random_state= 42)

model.fit(x_train_scaled,y_train)
y_preds = model.predict(x_valid_scaled)

In [11]:
# A plotting function
def plot_data(actual, predicted, title):

# model performance using RMSE
    rmse = np.sqrt(mean_squared_error(actual, predicted))

# R^2 (coefficient of determination) :
    R2 =r2_score(actual, predicted)
    plt.figure(figsize=(8,6))

# Plot the figure
    sn.regplot(x=predicted , y=actual,line_kws={"lw":2,
                                                "ls":"--",
                                                "color":"red",
                                                "alpha":0.7})
    plt.title(title, color="red")
    plt.xlabel("Predicted logS(mol/L)",
               color="blue")
    plt.xlim(-8,1)
    plt.ylabel("Experimental logS(mol/L)",
               color ="blue")

    plt.grid(alpha=0.3)
    R2 = mpatches.Patch(label="R2={:04.2f}".format(R2))
    rmse = mpatches.Patch(label="RMSE={:04.2f}".format(rmse))
    plt.legend(handles=[R2, rmse])

Let's plot the predicted logS of the validation set and see if our model works.

In [ ]:
sn.set_theme(style="whitegrid")
plot_data(y_valid,y_preds,"Validation data LGBM")

In [13]:
# Standard scaler - transform
x_scaled_test = scaler.transform(data_dl_descriptors)

# Predict solubility of the test data
y_test_preds = model.predict(x_scaled_test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [ ]:
# Plotting testing set
sn.set_theme(style="whitegrid")
plot_data(data_dl["LogS exp (mol/L)"], y_test_preds,
           "Test data: Drug-like Molecules")


## 7.2 XGB Regressor


In [15]:
from xgboost import XGBRegressor

model_XGB = XGBRegressor(n_estimators = 500,
                         max_depth = 5,
                         learning_rate = 0.1,
                         random_state = 42)
model_XGB.fit(x_train_scaled,y_train)
y_preds_XGB = model_XGB.predict(x_valid_scaled)

In [16]:
# A plotting function
def plot_data_XGB(actual, predicted, title):

# model performance using RMSE
    rmse_XGB = np.sqrt(mean_squared_error(actual, predicted))

# R^2 (coefficient of determination) :
    R2_XGB =r2_score(actual, predicted)
    plt.figure(figsize=(8,6))

# Plot the figure
    sn.regplot(x=predicted , y=actual,line_kws={"lw":2,
                                                "ls":"--",
                                                "color":"red",
                                                "alpha":0.7})
    plt.title(title, color="red")
    plt.xlabel("Predicted logS(mol/L)",
               color="blue")
    plt.xlim(-8,1)
    plt.ylabel("Experimental logS(mol/L)",
               color ="blue")


    plt.grid(alpha=0.3)
    R2_XGB = mpatches.Patch(label="R2_XGB={:04.2f}".format(R2_XGB))
    rmse_XGB = mpatches.Patch(label="RMSE_XGB={:04.2f}".format(rmse_XGB))
    plt.legend(handles=[R2_XGB, rmse_XGB])


Let's plot the predicted logS of the validation set and see if our model works.

In [ ]:
sn.set_theme(style="whitegrid")
plot_data_XGB(y_valid,y_preds_XGB,"Validation data XGB")

In [18]:
# Standard scaler - transform
x_scaled_test = scaler.transform(data_dl_descriptors)

# Predict solubility of the test data
y_test_preds_XGB = model_XGB.predict(x_scaled_test)

In [ ]:
# Plotting testing set for XGB
sn.set_theme(style="whitegrid")
plot_data_XGB(data_dl["LogS exp (mol/L)"], y_test_preds_XGB,
           "Test data XGB: Drug-like Molecules")


## 7.3 Random Forrest



Let's test the Random Forest model with optimized values we obtained.

In [45]:
# Replace NaN values by the average
imputer = SimpleImputer(strategy='mean')

# Imput the misssing values in x_train_scaled
x_train_imputed = imputer.fit_transform(x_train_scaled)

# Model creation for RandomForestRegressor
modelRF = RandomForestRegressor(n_estimators=901,
                               max_depth=26,
                               random_state=42)

# Model training on the given data
modelRF.fit(x_train_imputed, y_train)

# Imput missing values in x_valid_scaled
x_valid_imputed = imputer.transform(x_valid_scaled)

# Prediction
y_preds_RF = modelRF.predict(x_valid_imputed)


In [46]:
# A plotting function
def plot_data_RF(actual, predicted, title):

# model performance using RMSE
    rmse = np.sqrt(mean_squared_error(actual, predicted))

# R^2 (coefficient of determination) :
    R2 =r2_score(actual, predicted)
    plt.figure(figsize=(8,6))

# Plot the figure
    sn.regplot(x=predicted , y=actual,line_kws={"lw":2,
                                                "ls":"--",
                                                "color":"red",
                                                "alpha":0.7})
    plt.title(title, color="red")
    plt.xlabel("Predicted logS(mol/L)",
               color="blue")
    plt.xlim(-8,1)
    plt.ylabel("Experimental logS(mol/L)",
               color ="blue")


    plt.grid(alpha=0.3)
    R2 = mpatches.Patch(label="R2={:04.2f}".format(R2))
    rmse = mpatches.Patch(label="RMSE={:04.2f}".format(rmse))
    plt.legend(handles=[R2, rmse])

In [47]:
from sklearn.impute import SimpleImputer

# Calculate molecular descriptors for the test data
#Mol_descriptors_test, desc_names_test = RDkit_descriptors(data_dl["SMILES"])
#data_dl_descriptors = pd.DataFrame(Mol_descriptors_test, columns=desc_names_test)

# Impute missing values in the test data
imputer = SimpleImputer(strategy='mean')
x_scaled_test_imputed = imputer.fit_transform(data_dl_descriptors)

# Check the number of features in the training data (assuming it's the same as the model's expected features)
num_train_features = x_train_scaled.shape[1]

# If the number of features in the test data does not match the number of features in the training data,
# adjust the test data accordingly
if x_scaled_test_imputed.shape[1] != num_train_features:
    if x_scaled_test_imputed.shape[1] < num_train_features:
        # Add dummy columns for missing features
        num_missing_features = num_train_features - x_scaled_test_imputed.shape[1]
        dummy_columns = np.zeros((x_scaled_test_imputed.shape[0], num_missing_features))
        x_scaled_test_adjusted = np.hstack((x_scaled_test_imputed, dummy_columns))
    elif x_scaled_test_imputed.shape[1] > num_train_features:
        # Remove excess features
        x_scaled_test_adjusted = x_scaled_test_imputed[:, :num_train_features]
else:
    x_scaled_test_adjusted = x_scaled_test_imputed

# Predict solubility of the adjusted test data
y_test_preds = modelRF.predict(x_scaled_test_adjusted)

Let's plot the predicted logS of the validation set and see if our model works.

In [ ]:
# Plotting testing set
sn.set_theme(style="whitegrid")
plot_data_RF(data_dl["LogS exp (mol/L)"], y_test_preds,
           "Test data: Drug-like Molecules")

In [ ]:
# Standard scaler - transform
x_scaled_test = scaler.transform(data_dl_descriptors)

# Predict solubility of the test data
y_test_preds = modelRF.predict(x_scaled_test)


#8. Saving of the trained model and standard scaler

In [ ]:
#Save the model and the scaler using pickel
import pickle
with open("model_LGBM.pkl","wb") as f:
    pickle.dump(model,f)

with open("scaler_LGBM.pkl","wb") as f:
    pickle.dump(custom_scaler,f)

#9. Using the .pkl file

In [ ]:
import pickle
from sklearn import model # Import your specific model

# Load the pickle file
with open('your_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)